In [ ]:
import os, re
from pathlib import Path
import numpy as np
import pandas as pd

ROOT_DIR = Path("JATENG/JATENG")                 # extracted root folder
PROVINCE_NAME = "Jawa Tengah"              # e.g., "Jawa Timur" / "Jawa Tengah"
OUTPUT_FILE  = "Cleaned_Rainfall_JawaTengah.xlsx"
OUTPUT_FIXED = OUTPUT_FILE.replace(".xlsx", "_FIXED.xlsx")

MASTER_FILE = "Cleaned_Rainfall_Data_Java.xlsx" # optional master to append

def parse_meta_from_path(p: Path):
    """
    Extract (year, month, region) from a path like:
    .../<YEAR>/<REGION>/<MONTH>/file.xlsx
    - Year = first 4-digit '19xx'/'20xx' segment anywhere in the path
    - Month = closest folder above file that's an int 1..12
    - Region = the last non-numeric folder between Year and Month
    """
    parts = list(p.parts)

    year_idx, year_val = None, None
    for i, seg in enumerate(parts):
        m = re.fullmatch(r'(19|20)\d{2}', seg)
        if m:
            year_idx, year_val = i, int(seg)
            break

    month_idx, month_val = None, None
    for i in range(len(parts)-2, -1, -1):
        seg = parts[i]
        if seg.isdigit():
            v = int(seg)
            if 1 <= v <= 12:
                month_idx, month_val = i, v
                break

    region_val = None
    if year_idx is not None and month_idx is not None and month_idx > year_idx:
        span = parts[year_idx+1:month_idx]
        for seg in reversed(span):
            if seg and not seg.isdigit():
                region_val = seg.strip()
                break

    if not region_val:
        for seg in [p.parent.name, p.parent.parent.name if p.parent.parent else ""]:
            if seg and not seg.isdigit():
                region_val = seg.strip()
                break

    return year_val, month_val, region_val

def extract_rainfall_from_excel(path: str, province_hint: str):
    """
    BMKG sheet extraction:
      - find header row containing 'TANGGAL'
      - keep 'TANGGAL' + all 'RR' columns
      - cast to numeric; treat '-', 8888, 9999 as NaN
      - compute Avg_Rain_mm across stations
      - attach Province, Year, Month, Region from path
    Returns tidy df: [Province, Region, Year, Month, Date, Avg_Rain_mm]
    """
    try:
        raw = pd.read_excel(path, header=None)
    except Exception:
        return pd.DataFrame(columns=["Province","Region","Year","Month","Date","Avg_Rain_mm"])

    # locate the header row
    hdr = raw.index[
        raw.apply(lambda r: r.astype(str).str.contains(r'\bTANGGAL\b', case=False, na=False)).any(axis=1)
    ]
    if len(hdr) == 0:
        return pd.DataFrame(columns=["Province","Region","Year","Month","Date","Avg_Rain_mm"])
    start = int(hdr[0])

    df = pd.read_excel(path, header=start)
    df.columns = df.columns.astype(str).str.strip()
    if "TANGGAL" not in df.columns:
        df.rename(columns={df.columns[0]: "TANGGAL"}, inplace=True)

    # keep date + 'RR' cols
    keep = [c for c in df.columns if c == "TANGGAL" or "RR" in c]
    df = df[keep].copy()

    # numeric cleaning
    for c in df.columns:
        if c != "TANGGAL":
            df[c] = pd.to_numeric(df[c].replace({"-": np.nan}), errors="coerce")
            df[c].replace({8888: np.nan, 9999: np.nan}, inplace=True)

    rr_cols = [c for c in df.columns if c != "TANGGAL"]
    if not rr_cols:
        return pd.DataFrame(columns=["Province","Region","Year","Month","Date","Avg_Rain_mm"])

    df.rename(columns={"TANGGAL":"Date"}, inplace=True)
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce", dayfirst=True)
    df["Avg_Rain_mm"] = df[rr_cols].mean(axis=1, skipna=True)

    # attach path metadata (+ infer if missing)
    year, month, region = parse_meta_from_path(Path(path))
    if (year is None) and df["Date"].notna().any():
        ymode = df["Date"].dt.year.mode()
        if len(ymode): year = int(ymode.iloc[0])
    if (month is None) and df["Date"].notna().any():
        mmode = df["Date"].dt.month.mode()
        if len(mmode): month = int(mmode.iloc[0])

    tidy = df.dropna(subset=["Date"]).copy()
    tidy["Province"] = province_hint
    tidy["Region"] = region
    tidy["Year"] = year
    tidy["Month"] = month
    return tidy[["Province","Region","Year","Month","Date","Avg_Rain_mm"]]

excel_paths = list(ROOT_DIR.rglob("*.xlsx")) + list(ROOT_DIR.rglob("*.xls"))
harvest = []
for xp in excel_paths:
    out = extract_rainfall_from_excel(str(xp), PROVINCE_NAME)
    if not out.empty:
        harvest.append(out)

province_df = (pd.concat(harvest, ignore_index=True)
               if harvest else pd.DataFrame(columns=["Province","Region","Year","Month","Date","Avg_Rain_mm"]))

# basic clean
province_df["Avg_Rain_mm"] = pd.to_numeric(province_df["Avg_Rain_mm"], errors="coerce")
province_df["Year"] = pd.to_numeric(province_df["Year"], errors="coerce").astype("Int64")
province_df["Month"] = pd.to_numeric(province_df["Month"], errors="coerce").astype("Int64")
province_df["Date"] = pd.to_datetime(province_df["Date"], errors="coerce")
province_df["Province"] = province_df["Province"].astype(str).str.strip()
province_df["Region"] = province_df["Region"].astype(str).str.strip()

mask_numeric_region = province_df["Region"].str.fullmatch(r"\d+")
province_df = province_df[~mask_numeric_region].copy()

province_df = province_df.dropna(subset=["Date"]).sort_values(
    ["Province","Region","Year","Month","Date"]
).reset_index(drop=True)

# # ------------- SAVE SAFE COPY (FIXED) -------------
province_df.to_excel(OUTPUT_FIXED, index=False)
print(f"✅ Saved FIXED file: {OUTPUT_FIXED}")
print("Years:", sorted(province_df["Year"].dropna().unique().tolist()))
print("Regions (sample):", sorted(province_df["Region"].dropna().unique().tolist())[:20])
print(province_df.head(5))

if Path(MASTER_FILE).exists():
    base = pd.read_excel(MASTER_FILE)
    # ensure expected cols; add Province if master lacked it earlier
    if "Province" not in base.columns:
        base["Province"] = "Banten"
    base["Date"] = pd.to_datetime(base["Date"], errors="coerce", dayfirst=True)

    # normalize columns for concat (master may not have Month)
    base_cols = ["Region","Province","Year","Date","Avg_Rain_mm"]
    new_cols  = ["Region","Province","Year","Date","Avg_Rain_mm"]

    combined = (pd.concat([base[base_cols], province_df[new_cols]], ignore_index=True)
                  .sort_values(["Province","Region","Year","Date"]).reset_index(drop=True))
    combined.to_excel(MASTER_FILE, index=False)
    print(f"✅ Appended {PROVINCE_NAME} into {MASTER_FILE} | total rows: {len(combined)}")


✅ Saved FIXED file: Cleaned_Rainfall_JawaTengah_FIXED.xlsx
Years: [2020, 2021, 2022, 2023, 2024]
Regions (sample): ['Banjarnegara', 'Cilacap', 'Semarang', 'Tegal']
      Province        Region  Year  Month       Date  Avg_Rain_mm
0  Jawa Tengah  Banjarnegara  2020      1 2020-01-01         22.0
1  Jawa Tengah  Banjarnegara  2020      1 2020-01-02          NaN
2  Jawa Tengah  Banjarnegara  2020      1 2020-01-03          NaN
3  Jawa Tengah  Banjarnegara  2020      1 2020-01-04          NaN
4  Jawa Tengah  Banjarnegara  2020      1 2020-01-05          NaN
✅ Appended Jawa Tengah into Cleaned_Rainfall_Data_Java.xlsx | total rows: 19057
